Goal: pull out 1 buoy record that has ssc&chla and search for matching icesat2 granules

To do:
- Load info for 1 relevant station
- Find all matching atl granules and save times from file names
- Compare times to see if any matching buoy times

In [2]:
%pip install --quiet erddapy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np
from sliderule import sliderule, icesat2, earthdata

pd.set_option('display.max_columns', None)

sliderule.init()

Client (version (4, 5, 3)) is out of date with the server (version (4, 6, 2))


False

In [4]:
def fname2datetime(fname):
    y = int(fname[6:10])
    m = int(fname[10:12])
    d = int(fname[12:14])
    H = int(fname[14:16])
    M = int(fname[16:18])
    S = int(fname[18:20])

    t = datetime(y,m,d,H,M,S, tzinfo=timezone.utc)
    return t

def buoy_bound_box(lat,lon,buffer_km):
    # define a buffer distance around the buoy to search for icesat-2 data
    lat_buff = buffer_km/111 # convert buffer distance to frac of 1 deg lat
    lon_buff = buffer_km/(111*np.cos(lat*np.pi/180)) # convert buffer distance to frac of 1 deg lon
    # define bounding box around the buoy (WSEN)
    # example: bbox = [-108.3, 39.2, -107.8, 38.8]
    # bbox = [lon-lon_buff,lat+lat_buff,lon+lon_buff,lat-lat_buff]
    # region = sliderule.toregion(bbox)
    minx = lon - lon_buff
    miny = lat - lat_buff
    maxx = lon + lon_buff
    maxy = lat + lat_buff

    poly = [{'lon': minx, 'lat': miny},
            {'lon': maxx, 'lat': miny},
            {'lon': maxx, 'lat': maxy},
            {'lon': minx, 'lat': maxy},
            {'lon': minx, 'lat': miny}] # Closing the loop by repeating the first point
    return poly


In [5]:
# load a list of all the relevant ERDDAPs and their urls
FF = pd.read_pickle("labeled_relevant_stations.pkl")
FF[(FF.buoy == True) & (FF.ssc == True)]
FF.reset_index(drop=True, inplace=True)

5743


In [ ]:
# example with no looping (next cell is loops)
jj = 2

# make an ROI within a 0.1 km range of buoy
poly = buoy_bound_box(FF['geospatial_lat_min'][jj],
                      FF['geospatial_lon_min'][jj],
                      0.1)

# search CMR for ATL03 granules in the bounding box
grns = earthdata.cmr(short_name="ATL03",
                     polygon=poly,
                     version='006')

# save the times for each granule as a datetime object
icesat_times = [fname2datetime(fname) for fname in grns]
print(len(icesat_times))

# now check if buoy data exists for these granule times (loop)
kk = 5
t = icesat_times[kk]
filename = grns[kk]
print(filename)
# make erddap request for all data within +/- 1h of the icesat-2 granule
e = ERDDAP(server=FF['url'][jj],
           protocol="tabledap",
           response="csv")
e.dataset_id = FF['sites'][jj]
t_start = (t-timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%S+00:00")
t_end = (t+timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%S+00:00")
e.constraints = {"time>=": t_start,
                 "time<=": t_end}

# try to download the associated buoy data
buoy = e.to_pandas(parse_dates=True)

# try to download associated photons
t_start = (t-timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%SZ")
t_end = (t+timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%SZ")
poly = buoy_bound_box(FF['geospatial_lat_min'][jj],
                      FF['geospatial_lon_min'][jj],
                      2)
parms = {"poly": poly,
         "t0": t_start,
         "t1": t_end,
         "srt": icesat2.SRT_OCEAN}

atl_gdb = icesat2.atl03sp(parms)
len(atl_gdb)

In [ ]:
# loop through each buoy asset:
for jj in range(len(FF)):
    # define a search region around the buoy 
    lat = FF['geospatial_lat_min'][jj]
    lon = FF['geospatial_lon_min'][jj]
    region = buoy_bound_box(lat,lon,0.1)

    # search CMR for ATL03 granules in the bounding box
    grns = earthdata.cmr(short_name="ATL03",
                         polygon=region['poly'],
                         version='006')
    # save the times for each granule as a datetime object
    icesat_times = [fname2datetime(fname) for fname in grns]

    # now check if buoy data exists for these granules\n",
    e = ERDDAP(server=FF['url'][jj],
               protocol="tabledap",
               response="csv")
    e.dataset_id = FF['sites'][jj]

    for t in icesat_times:
        # add a time buffer (+/-1 hours) to search for relevant buoy data for each granule,
        t_start = (t-timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%S+00:00")
        t_end = (t+timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%S+00:00")
        e.constraints = {"time>=": t_start,
                        "time<=": t_end}

        # try to download the associated buoy data
        try:
            buoy = e.to_pandas(parse_dates=True)
        except:
            print('no buoy data for this granule')
            print('')
            continue
        # if buoy data exists, download the ATL03 photons in the bounding box at this time
        print('downloading ATL03')
        # Build ATL03 Request
        poly = buoy_bound_box(FF['geospatial_lat_min'][jj],
                              FF['geospatial_lon_min'][jj],
                              2)
        parms = {"poly": poly,
                 "t0": t_start,
                 "t1": t_end,
                 "srt": icesat2.SRT_OCEAN}        
        atl_gdb = icesat2.atl03sp(parms)
